In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
import sys
import glob
import openslide
import numpy as np
import pandas as pd
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter
from collections import defaultdict
from multiprocessing import Queue, Process
from multiprocessing import Pool, cpu_count
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
import logging

def prepare_logger(logger, level=logging.DEBUG):
    def prepare_handler(handler):
        handler.setLevel(level)
        handler.setFormatter(formatter)
        return handler

    formatter = logging.Formatter(fmt='%(asctime)s | [%(levelname)s] %(message)s',
                                 datefmt='%Y-%m-%d %H:%M:%S')

    del logger.handlers[:]
    logger.handlers.append(prepare_handler(logging.StreamHandler(sys.stderr)))

    logger.setLevel(level=level)
    return logger

In [4]:
import tensorflow as tf
tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

def build_model():
    inputs = [tf.keras.layers.Input(shape=(96,96,3)) for _ in range(3*3)]

    nasnet = tf.keras.applications.NASNetMobile(include_top=False, input_tensor=tf.keras.layers.Input(shape=(96,96,3)), weights='imagenet')
    nasnet.trainable = True

    gmax = tf.keras.layers.GlobalMaxPooling2D()
    gavg = tf.keras.layers.GlobalAveragePooling2D()
    flat = tf.keras.layers.Flatten()
    con = tf.keras.layers.Concatenate(axis=-1)
    drop = tf.keras.layers.Dropout(0.5)
    out = tf.keras.layers.Dense(1, activation='sigmoid')

    outs = []
    for inp in inputs:
        f = nasnet(inp)
        f1 = gmax(f)
        f2 = gavg(f)
        f3 = flat(f)
        f = con([f1,f2,f3])
        f = drop(f)
        f = out(f)
        outs.append(f)

    out = tf.keras.layers.Concatenate(axis=-1)(outs)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(out)
    model = tf.keras.Model(inputs, out)
    
    loss_fn = tf.keras.losses.binary_crossentropy
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
            tf.keras.metrics.Precision(), 
            tf.keras.metrics.Recall()]
    
    model.compile(loss=loss_fn, metrics=metrics, optimizer=tf.keras.optimizers.Adam(lr=0.0001))
    
    return model
model = build_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
def get_augmenter():
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5),  # horizontally flip 50% of all images
            iaa.Flipud(0.2),  # vertically flip 20% of all images
            sometimes(iaa.Affine(
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
                # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},  # translate by -20 to +20 percent (per axis)
                rotate=(-10, 10),  # rotate by -45 to +45 degrees
                shear=(-5, 5),  # shear by -16 to +16 degrees
                order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL  # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            # execute 0 to 5 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 5),
                    [
                        sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),
                        # convert images into their superpixel representation
                        iaa.OneOf([
                            iaa.GaussianBlur((0, 1.0)),  # blur images with a sigma between 0 and 3.0
                            iaa.AverageBlur(k=(3, 5)),
                            # blur image using local means with kernel sizes between 2 and 7
                            iaa.MedianBlur(k=(3, 5)),
                            # blur image using local medians with kernel sizes between 2 and 7
                        ]),
                        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)),  # sharpen images
                        iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),  # emboss images
                        # search either for all edges or for directed edges,
                        # blend the result with the original image using a blobby mask
                        iaa.SimplexNoiseAlpha(iaa.OneOf([
                            iaa.EdgeDetect(alpha=(0.5, 1.0)),
                            iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                        ])),
                        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01 * 255), per_channel=0.5),
                        # add gaussian noise to images
                        iaa.OneOf([
                            iaa.Dropout((0.01, 0.05), per_channel=0.5),  # randomly remove up to 10% of the pixels
                            iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
                        ]),
                        iaa.Invert(0.01, per_channel=True),  # invert color channels
                        iaa.Add((-2, 2), per_channel=0.5),
                        # change brightness of images (by -10 to 10 of original value)
                        iaa.AddToHueAndSaturation((-1, 1)),  # change hue and saturation
                        # either change the brightness of the whole image (sometimes
                        # per channel) or change the brightness of subareas
                        iaa.OneOf([
                            iaa.Multiply((0.9, 1.1), per_channel=0.5),
                            iaa.FrequencyNoiseAlpha(
                                exponent=(-1, 0),
                                first=iaa.Multiply((0.9, 1.1), per_channel=True),
                                second=iaa.ContrastNormalization((0.9, 1.1))
                            )
                        ]),
                        sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)),
                        # move pixels locally around (with random strengths)
                        sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
                        # sometimes move parts of the image around
                        sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                    ],
                    random_order=True
                    )
        ],
        random_order=True
    )
    return seq

def chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def divide_round_up(n, d):
        return (n + (d - 1))//d  
    
def calculate_metrics(y_pred, y_true):

    quality_metrics = {}

    # BINARY CROSSENTROPY
    m = tf.keras.losses.BinaryCrossentropy()
    quality_metrics['loss'] = m(y_true, y_pred)

    # BINARY ACCURACY
    m = tf.keras.metrics.BinaryAccuracy()
    m.update_state(y_true, y_pred)
    quality_metrics['accuracy'] = m.result().numpy()

    # PRECISION
    m = tf.keras.metrics.Precision()
    m.update_state(y_true, y_pred)
    quality_metrics['precision'] = m.result().numpy()

    # RECALL
    m = tf.keras.metrics.Recall()
    m.update_state(y_true, y_pred)
    quality_metrics['recall'] = m.result().numpy()

    # ROC-AUC
    m = roc_auc_score(y_true, y_pred)
    quality_metrics['auc'] = m

    return quality_metrics

In [14]:
def extract_slide_name_from_filename(filename, context=False):
    """ GET SLIDE NAME FROM FILENAME """
    parts = 2
    if context:
        parts = 3
    return os.path.splitext(os.path.basename(filename))[0].rsplit('-', parts)[0]

In [9]:
def get_slides(context_size, test_slides=False):
    """ READ VALID PANDAS COORDINATES """
    test_filter = '[!^T]'
    if test_slides:
        test_filter='T'
    
    
    if not context_size or context_size in [0,1]:
        return glob.glob(MAP_DIR + 'normal/{}*.gz'.filter(test_filter))
    else:
        return glob.glob(MAP_DIR + 'macro/{}*context{}.gz'.format(test_filter, context_size))
    
def open_all_slides(slides, context=False):
    """ OPEN EVERY SLIDE ON THE DISK """
    open_slides = {}
    for slide in slides:
        slide_name = extract_slide_name_from_filename(slide, context)
        slide_fn = SLIDE_DIR + slide_name + '.mrxs'
        open_slides[slide_name] = openslide.open_slide(slide_fn)
    return open_slides

In [23]:
def split_slides(slides, ratio=0.15):
    """ SPLIT SLIDES INTO TRAIN, VALID, TEST"""
    test_size = int(len(slides) * ratio)
    train_val, test = train_test_split(slides, test_size=test_size)
    train, valid = train_test_split(train_val, test_size=test_size)
    return train, valid, test

def get_sampler(slides, context=False):
    """ PREPARE SAMPLER """
    sampler = {'normal': {}, 'cancer': {}}
    for slide in slides:
        slide_name = extract_slide_name_from_filename(slide, context)
        df = pd.read_pickle(slide)
        
        normal_coords = df[df['class'] == 0]['class']
        cancer_coords = df[df['class'] == 1]['class']
        if len(normal_coords) > 0:
            sampler['normal'][slide_name] = normal_coords
        
        if len(cancer_coords) > 0:
            sampler['cancer'][slide_name] = cancer_coords
    return sampler

In [ ]:
""" GENERATE AMOUNT OF UNIQUE TRAIN SAMPLES """
def generate_train_epoch_coords(sampler, size):
    static_sample = defaultdict(set)
    for no in range(size):
        
        if (no+1) % 5000 == 0:
            print('{:,}\r'.format(no+1), end='')
        
        slide_name, coord, label = sample(sampler)
        while (coord, label) in static_sample[slide_name]:
            slide_name, coord, label = sample(sampler)
        
        static_sample[slide_name].add((coord, label))
    return static_sample

def sample(sampler):
    """ RETURN SINGLE VALID COORDINATE FROM SAMPLER """
    label = np.random.choice(list(sampler.keys()))
    slide_name = np.random.choice(list(sampler[label].keys()))
    row_idx = np.random.randint(low=0, high=len(sampler[label][slide_name]))
    row = sampler[label][slide_name].iloc[[row_idx]]
    return slide_name, row.index[0], row.values[0]

In [10]:
def extract_macrotiles(_input, verbose=True):
    slide_name, coord_set = _input
    log.debug('[WORKER] {:2} | PID #{}'.format(slide_name, os.getpid()))
    os_slide = slides[slide_name]
    
    result = {'fn': [], 'image': [], 'class': []}
    for coord, label in coord_set:
        result['fn'].append('{slide_name}-c{row}-r{col}'.format(slide_name=slide_name, col=coord[0], row=coord[1]))
        result['image'].append(cut_macrotile(extract_macrotile(os_slide, coord)))
        result['class'].append(label)
    log.debug('[WORKER] PID #{} DONE.'.format(os.getpid()))
    return pd.DataFrame.from_dict(result)

def extract_macrotile(os_slide, col_row):
    """ EXTRACT SINGLE TILE FROM A SLIDE """
    TILE_SIZE = 96
    MACROTILE_SIZE = 3*96
    LEVEL = 1
    
    col, row = col_row
    x_coord = (col-1)*TILE_SIZE
    y_coord = (row-1)*TILE_SIZE    
    return os_slide.read_region(location=(x_coord-TILE_SIZE, y_coord-TILE_SIZE), level=(LEVEL), size=(MACROTILE_SIZE,MACROTILE_SIZE)).convert('RGB')

def cut_macrotile(full_macrotile):
    CENTER_SIZE = 32
    TILE_SIZE = 96
    
    result = []
    for macro_x_coord in range(0,3*CENTER_SIZE,CENTER_SIZE):
        for macro_y_coord in range(0,3*CENTER_SIZE,CENTER_SIZE):
            result.append(np.array(full_macrotile.crop((macro_x_coord, macro_y_coord, macro_x_coord+TILE_SIZE, macro_y_coord+TILE_SIZE))))
    return np.array(result)

In [11]:
def prepare_dataset(ds_size, sampler, chunk_size=3200, batch_size=32, batch_queue=None, fn_queue=None):
    """  PROCESS TO PREPARE DATASET OF SIZE ds_size """
    log.debug('[PROCESS] PID: #{}'.format(os.getpid()))
    log.debug('[PROCESS] Preparing dataset')
    for _ in range(ds_size // chunk_size):
        generate_and_extract_tiles(sampler, chunk_size, batch_size, batch_queue, fn_queue)
    
    if ds_size % chunk_size > 0:
        generate_and_extract_tiles(sampler, ds_size % chunk_size, batch_size, batch_queue, fn_queue)
            
def generate_and_extract_tiles(sampler, chunk_size, batch_size, batch_queue, fn_queue):
    log.debug('[PROCESS] Prepating chunk...')
    chunk_coords = generate_train_epoch_coords(sampler, chunk_size)
    
    log.debug('[PROCESS] Pooling')
    with Pool(processes=10) as pool:
        result = pool.map(extract_macrotiles, chunk_coords.items())
        
    # Concatenate and shuffle
    log.debug('[PROCESS] Concatenating')
    df = pd.concat(result).sample(frac=1).reset_index(drop=True)
    
    log.debug('[PROCESS] Storing filenames')
    if fn_queue:
        fn_queue.put(df[['fn', 'class']].to_dict('list'))
    
    log.debug('[PROCESS] Returning')
    if batch_queue:
        for chunk in chunker(df[['image', 'class']], batch_size):
            batch_queue.put(chunk)
    else:
        return df
            
def calculate_total_steps(ds_size, chunk_size, batch_size):
    chunk_steps = divide_round_up(chunk_size, batch_size)
    last_steps = divide_round_up(ds_size % chunk_size, batch_size)
    no_chunks = ds_size // chunk_size
    return no_chunks * chunk_steps + last_steps
                              
def sequential_batch_generator(queue, steps):
    seq = get_augmenter()
    for _ in range(steps):
        batch_context = queue.get()
        data_batch = np.array([tf.keras.applications.nasnet.preprocess_input(seq.augment_images(batch)) for batch in batch_context['image']])
        data_batch = list(data_batch.transpose((1,0,2,3,4)))
        yield data_batch, batch_context['class'].values

In [12]:
def train(ds_size, sampler, batch_queue, chunk_size=3200, batch_size=32):
    metrics = basic_loop(ds_size, sampler, batch_queue, None, chunk_size, batch_size, False)
    print('Loss: {:.4f}'.format(metrics['loss']))
    return metrics
    
def evaluate(ds_size, sampler, batch_queue, eval_queue, chunk_size=3200, batch_size=32):
    metrics = basic_loop(ds_size, sampler, batch_queue, eval_queue, chunk_size, batch_size, True)                                        
    return metrics
                                         

def basic_loop(ds_size, sampler, batch_queue, eval_queue, chunk_size, batch_size, do_eval):
    log.debug('Preparing datagen.')
    
    p = Process(target=prepare_dataset, args=(ds_size, sampler, chunk_size, batch_size, batch_queue, eval_queue))
    p.start()

    steps = calculate_total_steps(ds_size, chunk_size, batch_size)
    datagen = sequential_batch_generator(batch_queue, steps)
    
    if do_eval:
        log.debug('Evaluating datagen')
        res = model.predict_generator(datagen, steps=steps, verbose=1)
        
        log.debug('Retrieving filenames and labels')
        #metadata_df = pd.concat([eval_queue.get() for _ in range(divide_round_up(ds_size, chunk_size))])
        metadata_df = [eval_queue.get() for _ in range(divide_round_up(ds_size, chunk_size))]
        metadata_df = pd.concat([pd.DataFrame(x) for x in metadata_df]).reset_index(drop=True)
        metadata_df['predict'] = res.ravel()
        
        predicts = metadata_df['predict'].ravel().astype(np.float)
        labels = metadata_df['class'].ravel().astype(np.float)
        
        metrics = calculate_metrics(predicts, labels)
    else:
        log.debug('Training model')
        history = model.fit_generator(datagen, steps_per_epoch=steps, verbose=1)
        metrics = {'loss': history.history['loss'][-1]}
    
    log.debug('Cleaning up')
    p.join()
    return metrics

def print_results(metrics):
    for k, v in metrics.items():
        print('| {:9} | {} |'.format(k.capitalize(),v))
        #print('| {:9} | {:.4f} |'.format(k.capitalize(),v))

In [14]:
SLIDE_DIR = '/mnt/data/scans/AI scans/Prostata/'
MAP_DIR = '/home/matejg/wsi_maps/Prostata/level1/'

In [24]:
""" PREPARING LOGGER"""
log = logging.getLogger()
log = prepare_logger(log, level=logging.DEBUG)

""" READ AND OPEN SLIDES"""
slide_coord_fn = get_slides(3)
slides = open_all_slides(slide_coord_fn, context=True)

""" SPLIT SLIDES """
train_fn, valid_fn, test_fn = split_slides(slide_coord_fn, ratio=0.15)

""" PREPARE SAMPLER """
train_sampler = get_sampler(train_fn, context=True)
valid_sampler = get_sampler(valid_fn, context=True)
test_sampler = get_sampler(test_fn, context=True)

In [31]:
%%time

TRAIN_DS_SIZE = 32*2000
VALID_DS_SIZE = 32*300
TEST_DS_SIZE = 32*300
CHUNK_SIZE = 3200
BATCH_SIZE = 32
MAX_QUEUE_SIZE = 100
NUM_EPOCHS = 2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 27.9 µs


In [ ]:
if __name__ == '__main__':
    batch_queue = Queue(MAX_QUEUE_SIZE)
    eval_queue = Queue(MAX_QUEUE_SIZE)
    
    for _ in range(2):
        print('Training')
        train(TRAIN_DS_SIZE, train_sampler, batch_queue, chunk_size=CHUNK_SIZE, batch_size=BATCH_SIZE)
    
        print('Validating')
        metrics = evaluate(VALID_DS_SIZE, valid_sampler, batch_queue, eval_queue, chunk_size=CHUNK_SIZE, batch_size=BATCH_SIZE)
        print_results(metrics)

print('Testing')
metrics = evaluate(TEST_DS_SIZE, test_sampler, batch_queue, eval_queue, chunk_size=CHUNK_SIZE, batch_size=BATCH_SIZE)
print_results(metrics)

2019-09-03 01:25:15 | [DEBUG] Preparing datagen.


Training


2019-09-03 01:25:15 | [DEBUG] Training model
2019-09-03 01:25:15 | [DEBUG] [PROCESS] PID: #1054
2019-09-03 01:25:15 | [DEBUG] [PROCESS] Preparing dataset
2019-09-03 01:25:15 | [DEBUG] [PROCESS] Prepating chunk...
2019-09-03 01:25:15 | [DEBUG] [PROCESS] Pooling
2019-09-03 01:25:16 | [DEBUG] [WORKER] M018-2012_1202-06-R | PID #1072
2019-09-03 01:25:16 | [DEBUG] [WORKER] M024-2012_0874-07-R | PID #1070
2019-09-03 01:25:16 | [DEBUG] [WORKER] M021-2012_1098-03-R | PID #1073
2019-09-03 01:25:16 | [DEBUG] [WORKER] M019-2012_1157-08-R | PID #1074
2019-09-03 01:25:16 | [DEBUG] [WORKER] M030-2012_1472-04-R | PID #1075
2019-09-03 01:25:16 | [DEBUG] [WORKER] M002-2012_2034-04-R | PID #1078
2019-09-03 01:25:16 | [DEBUG] [WORKER] M001-2012_1681-05-R | PID #1079
2019-09-03 01:25:16 | [DEBUG] [WORKER] M017-2012_4649-14-R | PID #1076
2019-09-03 01:25:16 | [DEBUG] [WORKER] M009-2012_2901-14-R | PID #1077
2019-09-03 01:25:16 | [DEBUG] [WORKER] M032-2012_2949-04-R | PID #1080
2019-09-03 01:25:16 | [DEBUG]

Instructions for updating:
Use tf.cast instead.


2019-09-03 01:27:21 | [WARNING] From /home/matejg/Test/test/lib/python3.5/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


...

2019-09-03 03:35:43 | [DEBUG] [PROCESS] Prepating chunk...
2019-09-03 03:35:44 | [DEBUG] [PROCESS] Pooling


1699/2000 [========================>.....] - ETA: 9:00 - loss: 0.6714 - binary_accuracy: 0.6369 - precision: 0.8654 - recall: 0.2952

2019-09-03 03:35:46 | [DEBUG] [WORKER] M018-2012_1202-06-R | PID #21871
2019-09-03 03:35:46 | [DEBUG] [WORKER] M021-2012_1098-03-R | PID #21872
2019-09-03 03:35:46 | [DEBUG] [WORKER] M019-2012_1157-08-R | PID #21873
2019-09-03 03:35:46 | [DEBUG] [WORKER] M032-2012_2949-04-R | PID #21880
2019-09-03 03:35:46 | [DEBUG] [WORKER] M017-2012_4649-14-R | PID #21881
2019-09-03 03:35:46 | [DEBUG] [WORKER] M024-2012_0874-07-R | PID #21870
2019-09-03 03:35:46 | [DEBUG] [WORKER] M030-2012_1472-04-R | PID #21883
2019-09-03 03:35:46 | [DEBUG] [WORKER] M009-2012_2901-14-R | PID #21882
2019-09-03 03:35:46 | [DEBUG] [WORKER] M031-2012_2263-05-R | PID #21885
2019-09-03 03:35:46 | [DEBUG] [WORKER] M001-2012_1681-05-R | PID #21886
2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21871 DONE.


1700/2000 [========================>.....] - ETA: 8:58 - loss: 0.6714 - binary_accuracy: 0.6369 - precision: 0.8655 - recall: 0.2952

2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21870 DONE.
2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21882 DONE.
2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21883 DONE.
2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21872 DONE.
2019-09-03 03:35:46 | [DEBUG] [WORKER] M002-2012_2034-04-R | PID #21871
2019-09-03 03:35:46 | [DEBUG] [WORKER] M026-2012_1102-08-R | PID #21882
2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21885 DONE.
2019-09-03 03:35:46 | [DEBUG] [WORKER] M008-2012_2101-04-R | PID #21870
2019-09-03 03:35:46 | [DEBUG] [WORKER] PID #21871 DONE.
2019-09-03 03:35:46 | [DEBUG] [WORKER] M014-2012_0381-04-R | PID #21883
2019-09-03 03:35:47 | [DEBUG] [WORKER] PID #21870 DONE.
2019-09-03 03:35:47 | [DEBUG] [WORKER] PID #21882 DONE.
2019-09-03 03:35:47 | [DEBUG] [WORKER] M029-2012_1819-10-R | PID #21872
2019-09-03 03:35:47 | [DEBUG] [WORKER] M022-2012_1285-07-R | PID #21871
2019-09-03 03:35:47 | [DEBUG] [WORKER] PID #21886 DONE.
2019-09-03 03:35:47 | [DEBUG] [WORKER] PID #21872 DONE.
2019-09-